### Create SPARK_HOME and PYLIB env var and update PATH env var

In [80]:
!pwd

/home/thomasj/Batch52


In [81]:
!ls /home/thomasj/Batch52

20190217_Batch52_CSE7322c_SparkML_Classification_Lab.ipynb  SparkGraphFrames
bank.csv


In [82]:
!hdfs dfs -ls /user/thomasj/bank.csv


-rw-r--r--   3 thomasj thomasj     375424 2019-02-11 14:28 /user/thomasj/bank.csv


In [83]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

### Initializing Spark

Build __SparkConf__ object 

    Contains information about your application.  


Create __SparkContext__ object 
    
    Tells Spark how to access a cluster. 
    

Create __SparkSession__ object

    The entry point to programming Spark with the Dataset and DataFrame API.

    Used to create DataFrame, register DataFrame as tables and execute SQL over tables etc.

In [84]:
# from pyspark.conf import SparkConf
# from pyspark import SparkContext
# from pyspark.sql import SparkSession

# conf = SparkConf().setAppName("Universal Bank Data Set").setMaster('local')
# sc = SparkContext(conf=conf)
# spark = SparkSession(sc)

In [85]:
# from pyspark.conf import SparkConf
# from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("SparkML")\
    .master('local[*]')\
    .getOrCreate()
sc = spark.sparkContext

### Loading the dependent libraries

In [86]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.sql.functions import isnan, when, count, col, countDistinct


#### Problem Statement
The dataset is from a bank, data related to direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, to access if the product (bank term deposit) would be (or not) subscribed. The data and attribute description are in the folder. 


#### Data Dictionary
 The dataset has the following attributes:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                    "blue-collar","self-employed","retired","technician","services") 

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

#### 6 - balance: average yearly balance, in euros (numeric) 

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")

9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  
15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

17 - Approved_no_yes - has the client subscribed to a __term deposit?__ (binary: "yes","no")

### Defining the schema to the data

In [87]:
## Define Schema
bankDataSchema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital_status", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),        
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("campaign", DoubleType(), True),
    StructField("pdays", DoubleType(), True),
    StructField("previous", DoubleType(), True),
    StructField("poutcome", StringType(), True),
    StructField("Approved_no_yes", StringType(), True)])

### Reading the data and creating a dataframe

In [88]:
## Read data and create a dataframe
#data = spark.read.format("csv")\
#       .option("header", "false")\
#       .option("inferSchema", "false")\
#       .load("/user/chaithanyas/bank.csv", schema = bankDataSchema)

In [89]:
!hdfs dfs -cat /user/thomasj/bank.csv | head -10











cat: Unable to write to output stream.


In [90]:
data = spark.read.csv(path='/user/thomasj/bank.csv',
                      header=False,
                      schema=bankDataSchema)

In [91]:
type(data)

pyspark.sql.dataframe.DataFrame

### Understanding Data

#### Print Schema

Hint: Use printSchema()

In [92]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- campaign: double (nullable = true)
 |-- pdays: double (nullable = true)
 |-- previous: double (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- Approved_no_yes: string (nullable = true)



#### Another way to check the data type of each attribute

Hint: Use dtypes

In [93]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital_status', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'double'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'double'),
 ('campaign', 'double'),
 ('pdays', 'double'),
 ('previous', 'double'),
 ('poutcome', 'string'),
 ('Approved_no_yes', 'string')]

#### Total number of Columns and Records

Hint: Use columns and count() function

In [94]:
print("No. of Columns = {}".format(len(data.columns)))

print('No. of Records = {}'.format(data.count()))

No. of Columns = 17
No. of Records = 4521


#### Look at first 3 row of the dataframe

In [95]:
data.head(3)

[Row(age=30, job=u'unemployed', marital_status=u'married', education=u'primary', default=u'no', balance=1787.0, housing=u'no', loan=u'no', contact=u'cellular', day=19, month=u'oct', duration=79.0, campaign=1.0, pdays=-1.0, previous=0.0, poutcome=u'unknown', Approved_no_yes=u'no'),
 Row(age=33, job=u'services', marital_status=u'married', education=u'secondary', default=u'no', balance=4789.0, housing=u'yes', loan=u'yes', contact=u'cellular', day=11, month=u'may', duration=220.0, campaign=1.0, pdays=339.0, previous=4.0, poutcome=u'failure', Approved_no_yes=u'no'),
 Row(age=35, job=u'management', marital_status=u'single', education=u'tertiary', default=u'no', balance=1350.0, housing=u'yes', loan=u'no', contact=u'cellular', day=16, month=u'apr', duration=185.0, campaign=1.0, pdays=330.0, previous=1.0, poutcome=u'failure', Approved_no_yes=u'no')]

Hint: Use Show() function

In [96]:
data.show(3)

+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|       job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
| 30|unemployed|       married|  primary|     no| 1787.0|     no|  no|cellular| 19|  oct|    79.0|     1.0| -1.0|     0.0| unknown|             no|
| 33|  services|       married|secondary|     no| 4789.0|    yes| yes|cellular| 11|  may|   220.0|     1.0|339.0|     4.0| failure|             no|
| 35|management|        single| tertiary|     no| 1350.0|    yes|  no|cellular| 16|  apr|   185.0|     1.0|330.0|     1.0| failure|             no|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----

#### Summary statistics

Hint: Use describe() function

In [97]:
data.describe().show()

+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|summary|               age|    job|marital_status|education|default|           balance|housing|loan| contact|               day|month|          duration|          campaign|             pdays|          previous|poutcome|Approved_no_yes|
+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|  count|              4521|   4521|          4521|     4521|   4521|              4521|   4521|4521|    4521|              4521| 4521|              4521|              4521|              4521|              4521|    4521|           4521|
|   mean| 41.17009511170095|   null|          null| 

Show only fixed set of colums 

In [98]:
data.describe().select('summary', 'age', 'loan', 'balance', 'pdays').show()

+-------+------------------+----+------------------+------------------+
|summary|               age|loan|           balance|             pdays|
+-------+------------------+----+------------------+------------------+
|  count|              4521|4521|              4521|              4521|
|   mean| 41.17009511170095|null|1422.6578190665782|39.766644547666445|
| stddev|10.576210958711263|null|3009.6381424673395|100.12112444301656|
|    min|                19|  no|           -3313.0|              -1.0|
|    max|                87| yes|           71188.0|             871.0|
+-------+------------------+----+------------------+------------------+



Observation

    Balance has -ve values

Count the number of records that have balance < 0

Hint: Use Where() and count()

In [99]:
data.where(data.balance < 0).count()

366

### Data Preprocessing

#### Replace negative balances with zeroes

Hint: Use withColumn() and when() functions

In [100]:
data = data.withColumn('balance', when(data.balance > 0, data.balance).otherwise(0))

In [101]:
data.where(data.balance < 0).count()

0

#### Handling missing values

Checking for null values at each column

In [102]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
             for c in data.columns]).show()

+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|job|marital_status|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|  0|  0|             0|        0|      0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|              0|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+



#### Split the data into training and test sets (30% held out for testing)

Hint: randomSplit()

In [103]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [104]:
testData.describe()

DataFrame[summary: string, age: string, job: string, marital_status: string, education: string, default: string, balance: string, housing: string, loan: string, contact: string, day: string, month: string, duration: string, campaign: string, pdays: string, previous: string, poutcome: string, Approved_no_yes: string]

In [105]:
# testData = testData.drop("loan")

#### Creating a list of categorical and numerical features

In [106]:
cat_Var_Names = ['job', 'marital_status', 'education', 'default', 'housing', 
                 'day', 'contact', 'month', 'poutcome', 'Approved_no_yes']

num_Var_Names = ['age', 'balance', 'duration', 'previous', 'pdays', 'campaign']

In [107]:
def generate_layout_bar(col_name):
    layout_bar = go.Layout(
        autosize=False, # auto size the graph? use False if you are specifying the height and width
        width=800, # height of the figure in pixels
        height=600, # height of the figure in pixels
        title = "Distribution of {} column".format(col_name), # title of the figure
        # more granular control on the title font 
        titlefont=dict( 
            family='Courier New, monospace', # font family
            size=14, # size of the font
            color='black' # color of the font
        ),
        # granular control on the axes objects 
        xaxis=dict( 
        tickfont=dict(
            family='Courier New, monospace', # font family
            size=14, # size of ticks displayed on the x axis
            color='black'  # color of the font
            )
        ),
        yaxis=dict(
#         range=[0,100],
            title='Percentage',
            titlefont=dict(
                size=14,
                color='black'
            ),
        tickfont=dict(
            family='Courier New, monospace', # font family
            size=14, # size of ticks displayed on the y axis
            color='black' # color of the font
            )
        ),
        font = dict(
            family='Courier New, monospace', # font family
            color = "white",# color of the font
            size = 12 # size of the font displayed on the bar
                )  
        )
    return layout_bar

In [108]:
import plotly.offline as pyoff
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [109]:
import numpy as np
from pyspark.sql import functions as F


In [110]:
def plot_bar(dataframe, col_name):
    # create a table with value counts
    temp =  trainingData.groupBy(col_name).count()
    temp = temp.toPandas()
    temp = temp.sort_values('count',ascending = False)
    temp[col_name] = temp[col_name].astype(str)
#     temp = dataframe[col_name].value_counts()
    # creating a Bar chart object of plotly
    data = [go.Bar(
            x=temp.loc[:,col_name].astype(str), # x axis values
            y=np.round(temp.loc[:,'count'].astype(float)/temp.loc[:,'count'].sum(),6)*100, # y axis values
            text = ['{}%'.format(i) for i in np.round(temp.loc[:,'count'].astype(float)/temp.loc[:,'count'].sum(),6)*100],
        # text to be displayed on the bar, we are doing this to display the '%' symbol along with the number on the bar
            textposition = 'auto', # specify at which position on the bar the text should appear
        marker = dict(color = '#0047AB'),)] # change color of the bar
    # color used here Cobalt Blue
     
    layout_bar = generate_layout_bar(col_name=col_name)

    fig = go.Figure(data=data, layout=layout_bar)
    return iplot(fig)
    

In [111]:
for i in cat_Var_Names:
        plot_bar(trainingData ,i)

#### Use VectorAssembler to combine a given list of numcolumns into a single vector column.

In [112]:
from pyspark.ml.feature import VectorAssembler

assembler_Num = VectorAssembler(inputCols=num_Var_Names, outputCol="num_features")

#### Scale all the numeric attributes using MinMaxScaler

    MinMaxScaler transforms a dataset of Vector rows, rescaling each feature to a specific range (often [0, 1]). 

    MinMaxScaler computes summary statistics on a data set and produces a MinMaxScalerModel. The model can then transform each feature individually such that it is in the given range.

In [113]:
from pyspark.ml.feature import MinMaxScaler

min_Max_Scalar = MinMaxScaler(inputCol="num_features", outputCol="scaled_num_features")

#### Covert categorical to numeric : 

    OneHotEncoder, StringIndexer, VectorAssembler

In [114]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

indexers_Cat = [StringIndexer(inputCol=cat_Var_Name, outputCol="{0}_index".format(cat_Var_Name)) for cat_Var_Name in cat_Var_Names ]
encoders_Cat = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_vec".format(indexer.getInputCol())) for indexer in indexers_Cat]
assembler_Cat = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders_Cat], outputCol="cat_features")

assembler = VectorAssembler(inputCols=["scaled_num_features", "cat_features"], outputCol="features")

In [115]:
indexer_Label = StringIndexer(inputCol="loan", outputCol="label")

In [116]:
preprocessiong_Stages = [assembler_Num]+[min_Max_Scalar]+indexers_Cat+encoders_Cat+[assembler_Cat]+[assembler]+[indexer_Label]

### Model Building and Evaluation

#### Logistic Regression

In [117]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, labelCol="label", featuresCol="features")

In [118]:
from pyspark.ml import Pipeline

lr_Pipeline = Pipeline(stages=preprocessiong_Stages+[lr]) 

lr_Pipeline_model = lr_Pipeline.fit(trainingData)

In [119]:
print("Coefficients: " + str(lr_Pipeline_model.stages[-1].coefficients))
print("Intercept: " + str(lr_Pipeline_model.stages[-1].intercept))

Coefficients: [-0.33300798650858077,-6.512745661462765,0.38482059671527413,0.8552523772452456,-0.03423933514430892,0.6383958405582143,0.15649085963873755,-0.0836143026183671,0.1350827303283566,0.3096771806011728,0.035069661611043974,0.064032151022977,0.05734300928032968,0.45406574713678516,-1.0556090045928674,-0.4401924463802292,-1.7508915003715535,-0.09487613908784427,-0.4837608128257156,0.40141909644462326,0.2280145979021185,0.18236785783984963,-1.1182459346354063,0.08281683196158961,-0.7950688713339552,-0.017248313197427374,-0.010032083821835678,0.04229075245921547,-0.5721585347081491,0.20885044619383636,0.2775061430322408,-0.1437060180730782,0.416722601148906,0.7961408894635188,-0.2354318106651081,0.28431603977047337,0.34552587188663453,0.1378126498507733,0.37778262040277544,-0.7536277986398041,0.21634919168423505,-0.06867915653658245,-0.44223073066899526,0.1374617698497519,0.0007442403761870578,0.602133638273788,-0.004044387682266872,0.3098339440936362,0.05768225301212102,-1.84959

In [120]:
lr_Summary = lr_Pipeline_model.stages[-1].summary
objectiveHistory = lr_Summary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
0.4328259763
0.42331668069
0.419087144183
0.401264827262
0.397232424549
0.395583407688
0.394208725962
0.393703030392
0.39344686149
0.393030439178
0.392511808553


In [121]:
train_predictions_lr = lr_Pipeline_model.transform(trainingData)
test_predictions_lr = lr_Pipeline_model.transform(testData)

In [122]:
test_predictions_lr.show(2,truncate=False)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+---------------------------------+-------------------------------------------------------------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+----------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------+-----------------------------------------+----------+
|age|job    |marital_status|education|default|balance|housing|loan|contact |day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|num_f

#### Evaluation : LR Model

In [123]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", 
                                              predictionCol="prediction",
                                              metricName="accuracy")

predictionAndLabels_train_lr = train_predictions_lr.select("prediction", "label")
train_accuracy_lr = evaluator.evaluate(predictionAndLabels_train_lr)

print("Train accuracy  = " + str(train_accuracy_lr))

predictionAndLabels_test_lr = test_predictions_lr.select("prediction", "label")
test_accuracy_lr = evaluator.evaluate(predictionAndLabels_test_lr)

print("Test accuracy = " + str(test_accuracy_lr))

Train accuracy  = 0.845047923323
Test accuracy = 0.856937455068


#### Tuning LR Model

In [124]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [125]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1]) \
    .addGrid(lr.elasticNetParam, [0.5])\
    .build()
    
lr_crossval = CrossValidator(estimator=lr_Pipeline,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=2)     

In [126]:
# Run cross-validation, and choose the best set of parameters.
lr_crossval_Model = lr_crossval.fit(trainingData)

In [127]:
train_predictions_lrcv = lr_crossval_Model.transform(trainingData)
test_predictions_lrcv = lr_crossval_Model.transform(testData)

In [128]:
predictionAndLabels_train_lrcv = train_predictions_lrcv.select("prediction", "label")
train_accuracycv = evaluator.evaluate(predictionAndLabels_train_lrcv)
print("Train set accuracy  = " + str(train_accuracycv))

predictionAndLabels_test_lrcv = test_predictions_lrcv.select("prediction", "label")
test_accuracycv = evaluator.evaluate(predictionAndLabels_test_lrcv)
print("Test set accuracy = " + str(test_accuracycv))

Train set accuracy  = 0.844089456869
Test set accuracy = 0.854061826024


#### Decision Tree Classifier

In [129]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [130]:
dt_Pipeline = Pipeline(stages=preprocessiong_Stages+[dt]) 

dt_Pipeline_model = dt_Pipeline.fit(trainingData)

In [131]:
train_predictions_dt = dt_Pipeline_model.transform(trainingData)
test_predictions_dt = dt_Pipeline_model.transform(testData)

In [132]:
test_predictions_dt.show(2)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----+-------------+--------------------+----------+
|age|    job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec|        

#### Evaluation : DT Model

In [133]:
predictionAndLabels_train_dt = train_predictions_dt.select("prediction", "label")
train_accuracy_dt = evaluator.evaluate(predictionAndLabels_train_dt)

print("Train accuracy  = " + str(train_accuracy_dt))

predictionAndLabels_test_dt = test_predictions_dt.select("prediction", "label")
test_accuracy_dt = evaluator.evaluate(predictionAndLabels_test_dt)

print("Test accuracy = " + str(test_accuracy_dt))

Train accuracy  = 0.848881789137
Test accuracy = 0.848310567937


#### Tuning DT Model

In [134]:
paramGridDT = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [1,6,10]) \
    .build()
    
dt_crossval = CrossValidator(estimator=dt_Pipeline,
                             estimatorParamMaps=paramGridDT,
                             evaluator=evaluator,
                             numFolds=2)     

In [135]:
# Run cross-validation, and choose the best set of parameters.
dt_crossval_Model = dt_crossval.fit(trainingData)

In [136]:
train_predictions_dtcv = dt_crossval_Model.transform(trainingData)
test_predictions_dtcv = dt_crossval_Model.transform(testData)

In [137]:
predictionAndLabels_train_dtcv = train_predictions_dtcv.select("prediction", "label")
train_accuracydtcv = evaluator.evaluate(predictionAndLabels_train_dtcv)
print("Train set accuracy  = " + str(train_accuracydtcv))

predictionAndLabels_test_dtcv = test_predictions_dtcv.select("prediction", "label")
test_accuracydtcv = evaluator.evaluate(predictionAndLabels_test_dtcv)
print("Test set accuracy = " + str(test_accuracydtcv))

Train set accuracy  = 0.844089456869
Test set accuracy = 0.854061826024


#### Random Forest Classifier

In [138]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [139]:
rf_Pipeline = Pipeline(stages=preprocessiong_Stages+[rf]) 

rf_Pipeline_model = rf_Pipeline.fit(trainingData)

In [140]:
train_predictions_rf = rf_Pipeline_model.transform(trainingData)
test_predictions_rf = rf_Pipeline_model.transform(testData)

In [141]:
test_predictions_rf.show(100)

+---+-------------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-------------+---------------+-------------+---------------+-------------+-------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|age|          job|marital_status|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_

#### Evaluation : RF Model

In [142]:
predictionAndLabels_train_rf = train_predictions_rf.select("prediction", "label")
train_accuracy_rf = evaluator.evaluate(predictionAndLabels_train_rf)

print("Train accuracy  = " + str(train_accuracy_rf))

predictionAndLabels_test_rf = test_predictions_rf.select("prediction", "label")
test_accuracy_rf = evaluator.evaluate(predictionAndLabels_test_rf)

print("Test accuracy = " + str(test_accuracy_rf))

Train accuracy  = 0.844408945687
Test accuracy = 0.854061826024


#### Tuning RF Model

In [143]:
paramGridRF = ParamGridBuilder()\
            .addGrid(rf.maxDepth, [5])\
            .addGrid(rf.numTrees, [20])\
            .build()
    
rf_crossval = CrossValidator(estimator=rf_Pipeline,
                             estimatorParamMaps=paramGridRF,
                             evaluator=evaluator,
                             numFolds=2)     

In [144]:
# Run cross-validation, and choose the best set of parameters.
rf_crossval_Model = rf_crossval.fit(trainingData)

In [145]:
train_predictions_rfcv = rf_crossval_Model.transform(trainingData)
test_predictions_rfcv = rf_crossval_Model.transform(testData)

In [146]:
predictionAndLabels_train_rfcv = train_predictions_rfcv.select("prediction", "label")
train_accuracyrfcv = evaluator.evaluate(predictionAndLabels_train_rfcv)
print("Train set accuracy  = " + str(train_accuracyrfcv))

predictionAndLabels_test_rfcv = test_predictions_rfcv.select("prediction", "label")
test_accuracyrfcv = evaluator.evaluate(predictionAndLabels_test_rfcv)
print("Test set accuracy = " + str(test_accuracyrfcv))

Train set accuracy  = 0.844408945687
Test set accuracy = 0.854061826024


#### Gradient Boosted Tree Classifier

In [147]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features")

In [148]:
gbt_Pipeline = Pipeline(stages=preprocessiong_Stages+[gbt]) 

gbt_Pipeline_model = gbt_Pipeline.fit(trainingData)

In [149]:
train_predictions_gbt = gbt_Pipeline_model.transform(trainingData)
test_predictions_gbt = gbt_Pipeline_model.transform(testData)

In [150]:
test_predictions_gbt.show(100, truncate= False)

+---+-------------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+---------------------------------+------------------------------------------------------------------------------------------------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-------------+---------------+-------------+---------------+-------------+-------------------+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------+-----------------------------------------+----------+
|age|job          |m

In [151]:
test_predictions_gbt.select("loan","prediction", "probability").show(100)

+----+----------+--------------------+
|loan|prediction|         probability|
+----+----------+--------------------+
|  no|       0.0|[0.89797325821152...|
|  no|       0.0|[0.91823954104733...|
|  no|       0.0|[0.91717311586025...|
|  no|       0.0|[0.91572865888822...|
|  no|       0.0|[0.93678701770956...|
|  no|       0.0|[0.88277027441944...|
| yes|       0.0|[0.93490254506717...|
|  no|       0.0|[0.92531960674125...|
|  no|       0.0|[0.91964935977249...|
|  no|       0.0|[0.85699619840384...|
|  no|       0.0|[0.86678115337963...|
|  no|       0.0|[0.80976458739476...|
|  no|       0.0|[0.92423624306891...|
|  no|       0.0|[0.85010149610643...|
|  no|       0.0|[0.87095025731305...|
|  no|       1.0|[0.34627779406234...|
| yes|       0.0|[0.90632024856506...|
|  no|       0.0|[0.80437438730709...|
| yes|       0.0|[0.73120079189622...|
| yes|       0.0|[0.88932869317189...|
| yes|       0.0|[0.85377596156639...|
|  no|       0.0|[0.92630060786837...|
|  no|       0.0|[0.89791

#### Evaluation : gbt Model

In [152]:
predictionAndLabels_train_gbt = train_predictions_gbt.select("prediction", "label")
train_accuracy_gbt = evaluator.evaluate(predictionAndLabels_train_gbt)

print("Train accuracy  = " + str(train_accuracy_gbt))

predictionAndLabels_test_gbt = test_predictions_gbt.select("prediction", "label")
test_accuracy_gbt = evaluator.evaluate(predictionAndLabels_test_gbt)

print("Test accuracy = " + str(test_accuracy_gbt))

Train accuracy  = 0.869009584665
Test accuracy = 0.847591660676


#### Tuning GBT Model

In [153]:
paramGridGBT = ParamGridBuilder()\
            .addGrid(gbt.maxDepth, [5])\
            .addGrid(gbt.maxIter, [20])\
            .addGrid(gbt.stepSize, [0.1])\
            .build()
    
gbt_crossval = CrossValidator(estimator=gbt_Pipeline,
                             estimatorParamMaps=paramGridGBT,
                             evaluator=evaluator,
                             numFolds=2)     

In [154]:
# Run cross-validation, and choose the best set of parameters.
gbt_crossval_Model = gbt_crossval.fit(trainingData)